# Rydberg Blockade Radius

In this notebook we probe the values of the interatomic distance for which we obtain a strong Rydberg Blockade effect.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip

In [ ]:
from pulser import Pulse, Sequence, Register
from pulser.devices import Chadoq2
from pulser.simulation import Simulation
from pulser.waveforms import BlackmanWaveform,ConstantWaveform

In [ ]:
duration = 1000 # Natural: ~1 µsec
pi_pulse = Pulse(BlackmanWaveform(duration,np.pi),ConstantWaveform(duration,0),0)
pi_pulse.draw()

In [ ]:
Omega_max = np.max(pi_pulse.amplitude.samples)
R_blockade = (5.008e6/Omega_max)**(1/6)
print(R_blockade)

In [ ]:
res = []
divisions = 5
distances = np.linspace(6.5,11.5,divisions)

In [ ]:
for i,R in enumerate(distances):
    print(f'{divisions-i} remaining...')
    # Atom Register and Device
    reg = Register.rectangle(1,2,spacing=R, prefix='atom')
    #reg.draw()

    # Pulse Sequence
    seq = Sequence(reg, Chadoq2)
    seq.declare_channel('ryd','rydberg_local','atom0')
    seq.declare_channel('ryd2','rydberg_local2','atom1')
    seq.add(pi_pulse,'ryd')
    seq.align('ryd','ryd2')
    seq.add(pi_pulse,'ryd2')
    #seq.draw()

    sim = Simulation(seq)
    print(sim._U/Omega_max)
    plt.show()
    #print(sim._hamiltonian.to_list())
    occup = qutip.tensor([qutip.qeye(2),qutip.basis(2,0)*qutip.basis(2,0).dag()])
    sim.run(obs_list=[occup])
    res.append(sim.output.expect[0])

In [ ]:
for i,R in enumerate(distances):
    plt.plot(res[i], label=f'R={R}')
    plt.legend()